# Hash Functions and Hash Tables

A hash table is a data structure to store data for fast searching. In particular, it is often implemented as an array of buckets holding your data and the data is indexed by hashing their keys using a hash function that converts a text string into an integer referred to as hash value. The hash value of the key modular the bucket size determines the index of the bucket to hold the data with that key.

In Python there’s no need to create your custom implementation of hash functions and hash tables since you may directly use built-in hash function hash(), and directly use dictionaries as hash tables. 

From the point of learning hash functions and hash tables, it helps to see Python code to create both.

In [46]:
import pprint

# define my own hash function: k mod m, 
# where k is the value of the key and m is the range of the hashed index

def myhash(key, size): # key is a text string
    hash_value = 7; # better use a prime number 
    for i in range(len(key)):
        hash_value = (hash_value * 31 + ord(key[i])) % size
        # better use a prime number 
    return hash_value 

In [47]:
class Hashtable:
    def __init__(self, dicts): # dicts is a dictionary of (key, value)
        self.bucket_size = len(dicts)
        self.buckets = [[] for _ in range(self.bucket_size)]
        self._assign_buckets(dicts)
        
    def _assign_buckets(self, dicts): # create a hash table for dicts
        for key in dicts:
            index = myhash(key, self.bucket_size)
            self.buckets[index].append([key, dicts[key]]) 

    def get_value(self, input_key): # search
        index = myhash(input_key, self.bucket_size)
        bucket = self.buckets[index]
        for key, value in bucket:
            if key == input_key:
                return(value)
        return None

    def __str__(self):
        return pprint.pformat(self.buckets) # here pformat is used to return a printable representation of the object

# open-addressing hash table using linear probing and quadratic probing
class OpenAddressingHashTable:
    def __init__(self, size=128, probing_method="linear"):
        self.table_size = size
        self.table = [None] * self.table_size  # array to store key-value pairs
        self.probing_method = probing_method   
        self.probing_steps = 0  # number of probing steps

    def _probe(self, hash_value, step):
        if self.probing_method == "linear":
            return (hash_value + step) % self.table_size
        elif self.probing_method == "quadratic":
            return (hash_value + step**2) % self.table_size

    def insert(self, key, value):
        hash_value = myhash(key, self.table_size)
        step = 0
        while True:
            index = self._probe(hash_value, step)
            self.probing_steps += 1  # Increment probing step counter
            if self.table[index] is None or self.table[index][0] == key:
                self.table[index] = (key, value)
                return
            step += 1
            if step >= self.table_size:  # Prevent infinite loop if table is full
                raise Exception("Hash table overflow")

    def get_probing_steps(self):
        return self.probing_steps

    def reset(self):
        self.table = [None] * self.table_size
        self.probing_steps = 0

In [35]:
if __name__ == "__main__":
     capitals = {
        'France': 'Paris',
        'United States': 'Washington D.C.',
        'Italy': 'Rome',
        'Canada': 'Ottawa'
     }


hashtable = Hashtable(capitals)
print(f'regular hash table')
print(hashtable)
print(f"The capital of Italy is {hashtable.get_value('Italy')}.")


open_hashtable = OpenAddressingHashTable(size=128, probing_method="linear")
    
for country, capital in capitals.items():
   open_hashtable.insert(country, capital)

print(f'Open addressing hash table:')
print(open_hashtable)
print(f"The capital of Italy is {open_hashtable.get_value('Italy')}.")

regular hash table
[[['United States', 'Washington D.C.'], ['Italy', 'Rome']],
 [['Canada', 'Ottawa']],
 [['France', 'Paris']],
 []]
The capital of Italy is Rome.
Open addressing hash table:
[('France', 'Paris'),
 ('Canada', 'Ottawa'),
 ('Italy', 'Rome'),
 ('United States', 'Washington D.C.')]
The capital of Italy is Rome.


The experiment below compares linear vs quadratic probing, by inserting 4-digit string keys into hash tables over 100 trials. Then tracking and comparing the average number of probing steps for each method.

In [66]:
import random

def main():
    num_trials = 100
    table_size = 256
    num_keys = 128
    linear_probing_steps = 0
    quadratic_probing_steps = 0

    for _ in range(num_trials):
        # Generate m unique 4-digit keys
        keys = set()
        while len(keys) < num_keys:
            keys.add(str(random.randint(1000, 9999))) # using 4 digit ints from 1000 to 9999 to replicate 4 digit codes. 

        # Initialize linear and quatratic probing hash tables
        linear_table = OpenAddressingHashTable(size=table_size, probing_method="linear")
        quadratic_table = OpenAddressingHashTable(size=table_size, probing_method="quadratic")

        # Linear
        for key in keys:
            linear_table.insert(key, "value")
        linear_probing_steps += linear_table.get_probing_steps()
        linear_table.reset() 

        # Quadratic
        for key in keys:
            quadratic_table.insert(key, "value")
        quadratic_probing_steps += quadratic_table.get_probing_steps()
        quadratic_table.reset() 

    # Calculate average number of probing steps
    avg_linear_probing_steps = linear_probing_steps / num_trials
    avg_quadratic_probing_steps = quadratic_probing_steps / num_trials

    print(f"Average probing steps for Linear: {avg_linear_probing_steps:.2f}")
    print(f"Average probing steps for Quadratic: {avg_quadratic_probing_steps:.2f}")

if __name__ == "__main__":
    main()

Average probing steps for Linear: 302.98
Average probing steps for Quadratic: 226.90
